In [ ]:
import os
import numpy as np
import pandas as pd
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
#train = pd.read_csv('/content/drive/MyDrive/data/train.csv')

train.head()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
pre = ImageDataGenerator(
    rescale = 1/255.,
    rotation_range = 15,
    width_shift_range=0.2, 
    height_shift_range=0.2,
    zoom_range = 0.2,
    shear_range=0.3,
    featurewise_center=False,
    featurewise_std_normalization=False,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split= 0.2,)
pos = ImageDataGenerator(rescale = 1./255)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mb = MultiLabelBinarizer().fit(train.labels.apply(lambda x : x.split()))
labels = pd.DataFrame(mb.transform(train.labels.apply(lambda x : x.split())), columns = mb.classes_)
new_train = pd.concat([train['image'], labels], axis=1)
new_train.head()

In [ ]:
seed = 88
target_size = (299,299)
batch_size = 32
resize_img = '../input/resized-plant2021/img_sz_384'
test_img = '../input/plant-pathology-2021-fgvc8/test_images'
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission.head()

In [ ]:
train_generator = pre.flow_from_dataframe(
                  new_train,
                  directory = resize_img, 
                  x_col = 'image',
                  y_col = new_train.columns.tolist()[1:],
                  subset = "training",
                  color_mode = "rgb",
                  target_size = target_size,
                  class_mode = "raw",
                  batch_size = batch_size,
                  shuffle = True,
                  seed = seed,)
valid_generator = pre.flow_from_dataframe(
                  new_train,
                  directory = resize_img,
                  x_col = 'image',
                  y_col = new_train.columns.tolist()[1:],
                  subset = "validation",
                  color_mode = "rgb",
                  target_size = target_size,
                  class_mode = "raw",
                  batch_size = batch_size,
                  shuffle = True,
                  seed = seed,)
test_generator = pos.flow_from_dataframe(
                  submission,
                  directory = test_img,
                  x_col = 'image',
                  y_col = 'labels',
                  class_mode = None,
                  batch_size=8,
                  target_size = target_size,
                  color_mode="rgb",
                  shuffle = False,
                  seed = seed,)

In [ ]:
import matplotlib.pyplot as plt 
example = next(train_generator)
print(example[0].shape) 
plt.imshow(example[0][0,:,:,:]) 
plt.show()

In [ ]:
import keras
def load_model():
    model_pre = keras.models.load_model("../input/inceptionv3h5/299x299_V3.h5")
    
    return model_pre
model = load_model()

In [ ]:
model = load_model()
pred = model.predict(test_generator)

In [ ]:
perdict = (pred>0.30)
n_label = new_train.columns.tolist()[1:]
answer = []

for i in range(perdict.shape[0]):
    temp = []
    for j, k in enumerate(n_label):
        if perdict[i, j]:
            temp.append(k)
    answer.append(temp)
    
answer = [' '.join(n) for n in answer]

In [ ]:
submission['labels'] = np.array(answer)
submission

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
# !mv /content/V3.h5 /content/drive/MyDrive/apple